In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb

In [10]:
play_by_play = pd.read_csv('/Users/seandiehl/Desktop/NBA_PBP_2017-18.csv')
pd.set_option('display.max_columns', None)

In [11]:
play_by_play.head()

,URL,GameType,Location,Date,Time,WinningTeam,Quarter,SecLeft,AwayTeam,AwayPlay,AwayScore,HomeTeam,HomePlay,HomeScore,Shooter,ShotType,ShotOutcome,ShotDist,Assister,Blocker,FoulType,Fouler,Fouled,Rebounder,ReboundType,ViolationPlayer,ViolationType,TimeoutTeam,FreeThrowShooter,FreeThrowOutcome,FreeThrowNum,EnterGame,LeaveGame,TurnoverPlayer,TurnoverType,TurnoverCause,TurnoverCauser,JumpballAwayPlayer,JumpballHomePlayer,JumpballPoss
0,/boxscores/201710170CLE.html,regular,Quicken Loans Arena Cleveland Ohio,October 17 2017,8:01 PM,CLE,1,720,BOS,Jump ball: K. Love vs. A. Horford (K. Irving g...,0,CLE,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,K. Love - loveke01,A. Horford - horfoal01,K. Irving - irvinky01
1,/boxscores/201710170CLE.html,regular,Quicken Loans Arena Cleveland Ohio,October 17 2017,8:01 PM,CLE,1,704,BOS,K. Irving makes 2-pt jump shot from 10 ft (ass...,2,CLE,NaN,0,K. Irving - irvinky01,2-pt jump shot,make,10.0,A. Horford - horfoal01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,/boxscores/201710170CLE.html,regular,Quicken Loans Arena Cleveland Ohio,October 17 2017,8:01 PM,CLE,1,687,BOS,NaN,2,CLE,D. Rose misses 2-pt layup from 1 ft (block by ...,0,D. Rose - rosede01,2-pt layup,miss,1.0,NaN,A. Horford - horfoal01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,/boxscores/201710170CLE.html,regular,Quicken Loans Arena Cleveland Ohio,October 17 2017,8:01 PM,CLE,1,683,BOS,Defensive rebound by A. Horford,2,CLE,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A. Horford - horfoal01,defensive,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,/boxscores/201710170CLE.html,regular,Quicken Loans Arena Cleveland Ohio,October 17 2017,8:01 PM,CLE,1,681,BOS,G. Hayward misses 3-pt jump shot from 25 ft,2,CLE,NaN,0,G. Hayward - haywago01,3-pt jump shot,miss,25.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
play_by_play['Shooter'] = play_by_play['Shooter'].str.split('- ').str[1]
play_by_play.head()

,URL,GameType,Location,Date,Time,WinningTeam,Quarter,SecLeft,AwayTeam,AwayPlay,AwayScore,HomeTeam,HomePlay,HomeScore,Shooter,ShotType,ShotOutcome,ShotDist,Assister,Blocker,FoulType,Fouler,Fouled,Rebounder,ReboundType,ViolationPlayer,ViolationType,TimeoutTeam,FreeThrowShooter,FreeThrowOutcome,FreeThrowNum,EnterGame,LeaveGame,TurnoverPlayer,TurnoverType,TurnoverCause,TurnoverCauser,JumpballAwayPlayer,JumpballHomePlayer,JumpballPoss
0,/boxscores/201710170CLE.html,regular,Quicken Loans Arena Cleveland Ohio,October 17 2017,8:01 PM,CLE,1,720,BOS,Jump ball: K. Love vs. A. Horford (K. Irving g...,0,CLE,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,K. Love - loveke01,A. Horford - horfoal01,K. Irving - irvinky01
1,/boxscores/201710170CLE.html,regular,Quicken Loans Arena Cleveland Ohio,October 17 2017,8:01 PM,CLE,1,704,BOS,K. Irving makes 2-pt jump shot from 10 ft (ass...,2,CLE,NaN,0,irvinky01,2-pt jump shot,make,10.0,A. Horford - horfoal01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,/boxscores/201710170CLE.html,regular,Quicken Loans Arena Cleveland Ohio,October 17 2017,8:01 PM,CLE,1,687,BOS,NaN,2,CLE,D. Rose misses 2-pt layup from 1 ft (block by ...,0,rosede01,2-pt layup,miss,1.0,NaN,A. Horford - horfoal01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,/boxscores/201710170CLE.html,regular,Quicken Loans Arena Cleveland Ohio,October 17 2017,8:01 PM,CLE,1,683,BOS,Defensive rebound by A. Horford,2,CLE,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A. Horford - horfoal01,defensive,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,/boxscores/201710170CLE.html,regular,Quicken Loans Arena Cleveland Ohio,October 17 2017,8:01 PM,CLE,1,681,BOS,G. Hayward misses 3-pt jump shot from 25 ft,2,CLE,NaN,0,haywago01,3-pt jump shot,miss,25.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
play_by_play['Assister'] = play_by_play['Assister'].str.split('- ').str[1]
play_by_play['Blocker'] = play_by_play['Blocker'].str.split('- ').str[1]
play_by_play['Fouler'] = play_by_play['Fouler'].str.split('- ').str[1]
play_by_play['Fouled'] = play_by_play['Fouled'].str.split('- ').str[1]
play_by_play['Rebounder'] = play_by_play['Rebounder'].str.split('- ').str[1]
play_by_play['ViolationPlayer'] = play_by_play['ViolationPlayer'].str.split('- ').str[1]
play_by_play['FreeThrowShooter'] = play_by_play['FreeThrowShooter'].str.split('- ').str[1]
play_by_play['EnterGame'] = play_by_play['EnterGame'].str.split('- ').str[1]
play_by_play['LeaveGame'] = play_by_play['LeaveGame'].str.split('- ').str[1]
play_by_play['TurnoverPlayer'] = play_by_play['TurnoverPlayer'].str.split('- ').str[1]
play_by_play['TurnoverCauser'] = play_by_play['TurnoverCauser'].str.split('- ').str[1]

In [14]:
play_by_play.head()

,URL,GameType,Location,Date,Time,WinningTeam,Quarter,SecLeft,AwayTeam,AwayPlay,AwayScore,HomeTeam,HomePlay,HomeScore,Shooter,ShotType,ShotOutcome,ShotDist,Assister,Blocker,FoulType,Fouler,Fouled,Rebounder,ReboundType,ViolationPlayer,ViolationType,TimeoutTeam,FreeThrowShooter,FreeThrowOutcome,FreeThrowNum,EnterGame,LeaveGame,TurnoverPlayer,TurnoverType,TurnoverCause,TurnoverCauser,JumpballAwayPlayer,JumpballHomePlayer,JumpballPoss
0,/boxscores/201710170CLE.html,regular,Quicken Loans Arena Cleveland Ohio,October 17 2017,8:01 PM,CLE,1,720,BOS,Jump ball: K. Love vs. A. Horford (K. Irving g...,0,CLE,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,K. Love - loveke01,A. Horford - horfoal01,K. Irving - irvinky01
1,/boxscores/201710170CLE.html,regular,Quicken Loans Arena Cleveland Ohio,October 17 2017,8:01 PM,CLE,1,704,BOS,K. Irving makes 2-pt jump shot from 10 ft (ass...,2,CLE,NaN,0,irvinky01,2-pt jump shot,make,10.0,horfoal01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,/boxscores/201710170CLE.html,regular,Quicken Loans Arena Cleveland Ohio,October 17 2017,8:01 PM,CLE,1,687,BOS,NaN,2,CLE,D. Rose misses 2-pt layup from 1 ft (block by ...,0,rosede01,2-pt layup,miss,1.0,NaN,horfoal01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,/boxscores/201710170CLE.html,regular,Quicken Loans Arena Cleveland Ohio,October 17 2017,8:01 PM,CLE,1,683,BOS,Defensive rebound by A. Horford,2,CLE,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,horfoal01,defensive,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,/boxscores/201710170CLE.html,regular,Quicken Loans Arena Cleveland Ohio,October 17 2017,8:01 PM,CLE,1,681,BOS,G. Hayward misses 3-pt jump shot from 25 ft,2,CLE,NaN,0,haywago01,3-pt jump shot,miss,25.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
play_by_play.drop(columns=['URL', 'GameType', 'Location', 'Time', 'JumpballAwayPlayer', 'JumpballHomePlayer', 'JumpballPoss', 'TimeoutTeam'], inplace=True)

In [20]:
play_by_play.head()

,Date,WinningTeam,Quarter,SecLeft,AwayTeam,AwayPlay,AwayScore,HomeTeam,HomePlay,HomeScore,Shooter,ShotType,ShotOutcome,ShotDist,Assister,Blocker,FoulType,Fouler,Fouled,Rebounder,ReboundType,ViolationPlayer,ViolationType,FreeThrowShooter,FreeThrowOutcome,FreeThrowNum,EnterGame,LeaveGame,TurnoverPlayer,TurnoverType,TurnoverCause,TurnoverCauser
0,October 17 2017,CLE,1,720,BOS,Jump ball: K. Love vs. A. Horford (K. Irving g...,0,CLE,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,October 17 2017,CLE,1,704,BOS,K. Irving makes 2-pt jump shot from 10 ft (ass...,2,CLE,NaN,0,irvinky01,2-pt jump shot,make,10.0,horfoal01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,October 17 2017,CLE,1,687,BOS,NaN,2,CLE,D. Rose misses 2-pt layup from 1 ft (block by ...,0,rosede01,2-pt layup,miss,1.0,NaN,horfoal01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,October 17 2017,CLE,1,683,BOS,Defensive rebound by A. Horford,2,CLE,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,horfoal01,defensive,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,October 17 2017,CLE,1,681,BOS,G. Hayward misses 3-pt jump shot from 25 ft,2,CLE,NaN,0,haywago01,3-pt jump shot,miss,25.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
